Visualisasi Data Teks pada Topik Formula E di Twitter

In [ ]:
#Import module-module yang dibutuhkan

import warnings; warnings.simplefilter('ignore')
import nltk

try:
    import google.colab
    IN_COLAB = True
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudataSNA.py
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudata_sna.py
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/network_unp2021.gexf
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudataDDGsna.py
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/lib/taudataNlpTm.py
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudataEDA02CTA.py
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/contoh.pdf
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/slang.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/stopwords_id.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/stopwords_en.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/kata_dasar.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/wn-ind-def.tab
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/wn-msa-all.tab
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/ind_SA.csv
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/all_indo_man_tag_corpus_model.crf.tagger
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/corpus_sederhana.txt

    !pip install --upgrade spacy python-crfsuite unidecode textblob sastrawi sklearn-pycrfsuite
    !pip install --upgrade unidecode twython tweepy beautifulsoup4 json_lines lxml pyLDAvis json_lines
    !pip install --upgrade spacy python-crfsuite unidecode textblob sastrawi sklearn-pycrfsuite
    !pip install --upgrade unidecode twython tweepy beautifulsoup4 tika
    !python -m spacy download en
    !python -m spacy download xx
    !python -m spacy download en_core_web_sm
    nltk.download('popular')
except:
    IN_COLAB = False
    print("Running the code locally, please make sure all the python module versions agree with colab environment and all data/assets downloaded")

import taudataDDGsna as tau
import tweepy, json, urllib.request, requests
from urllib.request import Request, urlopen
from twython import TwythonStreamer
from bs4 import BeautifulSoup as bs
from tqdm import tqdm

--2022-06-11 11:57:50--  https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudataSNA.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-06-11 11:57:50 ERROR 404: Not Found.

--2022-06-11 11:57:50--  https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudata_sna.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20088 (20K) [text/plain]
Saving to: ‘taudata_sna.py.1’

taudata_sna.py.1    100%[===================>]  19.62K  --.-KB/s    in 0.001s  

2022-06-11 11:57:51 (30.2 MB/s) - ‘taudata_s

In [ ]:
# masukkan key-key dari twitter API
Ck = 'Vf77HCiDxhSQajsiql95M1czB' # consumer_key
Cs = 'QOWSiNa93GF29Hju8DCZ25THIC23YT3BEMU3bNywyL0mZZPvgh' # consumer_secret
At = '838209832570138624-UIJFno4pwB0HTAqNVZgblmYFxcDkaka' # access_token
As = 'TOSKn8SIYb0aufseN92DYVXCtcLGBBKpJba4fyGfj70G8' # access_secret

In [ ]:
#Login dengan API key barusan

auth = tweepy.auth.OAuthHandler(Ck, Cs)
auth.set_access_token(At, As)
api = tweepy.API(auth, wait_on_rate_limit=True, timeout=180, retry_count=5, retry_delay=3)
usr_ = api.verify_credentials()
print('Welcome "{}" you are now connected to twitter server'.format(usr_.name))

Welcome "Arel" you are now connected to twitter server


In [ ]:
qry = 'formula e' #karena akan digunakan bahasa Indonesia untuk mencari tweetnya, otomatis semua keyword formula e pasti membahas formula e jakarta yang baru saja dilaksanakan

In [ ]:
N = 5000 # jumlah N tweet "terkini" yang ingin diambil
T = []
for post in tqdm(tweepy.Cursor(api.search_tweets, q=qry, lang='id', tweet_mode='extended').items(N)):
    T.append(post)

2686it [00:49, 60.62it/s]Rate limit reached. Sleeping for: 851
2686it [01:00, 60.62it/s]

In [ ]:
# Jumlah tweets yang diperoleh
Tweet = [t._json for t in T]
print(' Berhasil mendapatkan {} tweets'.format(len(T)))

Berhasil mendapatkan 5000 tweets


In [ ]:
Tweet[0]['full_text'] # melihat tweet pertama yang diambil

@kurangbayar Ini linknya
Prof. Refly Harun:
*Satu di Anak Emaskan tapi  Gagal Total di MotoGP Mandalika.*

Satunya Lagi..,
*Anies Baswedan di anggap Anak Pinggiran yang Sukses Besar Menggelar Event Internasional Balap Mobil Formula 

https://t.co/9DkmvdmCFi

🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨
.


In [ ]:
# Fungsi untuk menyimpan tweetsnya
def saveTweets(tweets, file='Tweets.json'): #in Json Format
    with open(file, 'w') as f:
        for t in tweets:
            try:
                f.write(json.dumps(t)+'\n')
            except:
                pass

In [ ]:
# Menyimpan hasil crawling twitter secara utuh
fileName = 'Tweets_FormulaE.json'
saveTweets(Tweet, file=fileName)
print('Saved to '+fileName)

Saved to Tweets_FormulaE.json


In [ ]:
# Fungsi untuk load kembali
def loadTweets(file='Tweets.json'):
    f=open(file,encoding='utf-8', errors ='ignore', mode='r')
    T=f.readlines();f.close()
    for i,t in enumerate(T):
        T[i] = json.loads(t.strip())
    return T

In [ ]:
# Me-load kembali jika analisa ingin dilakukan di lain waktu
# Nama variabelnya dibedakan (T2)
fileName = 'Tweets_FormulaE.json'
T2 = loadTweets(file=fileName)
print('tweet pertama oleh "{}" : "{}"'.format(T2[0]['user']['screen_name'],T2[0]['full_text']))

tweet pertama oleh "TSolihien" : "@kurangbayar Ini linknya
Prof. Refly Harun:
*Satu di Anak Emaskan tapi  Gagal Total di MotoGP Mandalika.*

Satunya Lagi..,
*Anies Baswedan di anggap Anak Pinggiran yang Sukses Besar Menggelar Event Internasional Balap Mobil Formula 

https://t.co/9DkmvdmCFi

🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨
."


In [ ]:
# Mengambil hanya data tweet
Tweets_saja = [t['full_text'] for t in T2]
Tweets_saja[:5] # 5 tweet pertama

['@kurangbayar Ini linknya\nProf. Refly Harun:\n*Satu di Anak Emaskan tapi  Gagal Total di MotoGP Mandalika.*\n\nSatunya Lagi..,\n*Anies Baswedan di anggap Anak Pinggiran yang Sukses Besar Menggelar Event Internasional Balap Mobil Formula \n\nhttps://t.co/9DkmvdmCFi\n\n🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨\n.',
 'RT @OPPOSISINKRI: @UyokBack Ternyata dipake juga buat demo usut formula E \nhttps://t.co/9tDnycDfFu',
 'RT @propaganda_99: https://t.co/w69lX2H7X8\n\nhttps://t.co/s03YP3y3j3\n\nhttps://t.co/LQL8uoPdGk\n\nB 9532 MW mobil yg sama dipakai di bbrp aksi…',
 'RT @tatakujiyati: Keberhasilan negosiasi kita. Formula E mau tuh menghormati nilai2 lokal Indonesia: tak ada sampanye sbg selebrasi kemenan…',
 'RT @propaganda_99: https://t.co/w69lX2H7X8\n\nhttps://t.co/s03YP3y3j3\n\nhttps://t.co/LQL8uoPdGk\n\nB 9532 MW mobil yg sama dipakai di bbrp aksi…']

## Cleaning Data ##
1. Menghapus link url (biasanya link berita)
2. Menghapus username pada tweet
3. Mengubah hashtags dalam tweet menjadi tulisan
4. Menghapus non alpha-numeric (terutama emote)
5. Tokenisasi dan Menangani slang atau singkatan di data teks
6. Filtering/menggunakan Stopwords untuk menghilangkan kata-kata yang sering muncul


In [ ]:
# Menghapus link url pada tweet
import re
urlPattern = re.compile(r'https:\/\/t.co\/[a-zA-Z0-9\-\.]{0,10}')

T2=' '.join([str(item) for item in Tweets_saja])
T3=re.sub(urlPattern,' ',T2)
print(T3)# Tweet tanpa URL

URLs = re.findall(urlPattern,T2)# URL yang dihapus
print('URL yang ditemukan: ',str(URLs))

@kurangbayar Ini linknya
Prof. Refly Harun:
*Satu di Anak Emaskan tapi  Gagal Total di MotoGP Mandalika.*

Satunya Lagi..,
*Anies Baswedan di anggap Anak Pinggiran yang Sukses Besar Menggelar Event Internasional Balap Mobil Formula 

 

🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨
. RT @OPPOSISINKRI: @UyokBack Ternyata dipake juga buat demo usut formula E 
  RT @propaganda_99:  

 

 

B 9532 MW mobil yg sama dipakai di bbrp aksi… RT @tatakujiyati: Keberhasilan negosiasi kita. Formula E mau tuh menghormati nilai2 lokal Indonesia: tak ada sampanye sbg selebrasi kemenan… RT @propaganda_99:  

 

 

B 9532 MW mobil yg sama dipakai di bbrp aksi… RT @geloraco: Formula E Disebut Bikin Ancol Rugi, Geisz Ingatkan Proyek LRT: Biaya Rp6,5 T Masuk Cuma Rp500 Juta
  RT @BossTemlen: Nafas nyesek, mata kunang-kunang
tidur gelisah, pikiran kalut, perut mual, makan gak enak, itulah gambaran kondisi @GunRoml… RT @GeiszChalifah: Formula E menyingkap Gorden, antara Petarumg dengan Pecundang. RT @maspiyuaja: Buzzer Genjer STRESSSSSS g

In [ ]:
# Menghapus Username pada tweet
unamePattern = re.compile(r'^@[\w_:]{1,15}|@[\w]{1,15}')

T4=re.sub(unamePattern,' ',T3)
print(T4)# Tweet tanpa username

Usernames = re.findall(unamePattern,T3)# Username yang dihapus
print('Username yang ditemukan: ',str(Usernames))

  Ini linknya
Prof. Refly Harun:
*Satu di Anak Emaskan tapi  Gagal Total di MotoGP Mandalika.*

Satunya Lagi..,
*Anies Baswedan di anggap Anak Pinggiran yang Sukses Besar Menggelar Event Internasional Balap Mobil Formula 

 

🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨
. RT  :   Ternyata dipake juga buat demo usut formula E 
  RT  :  

 

 

B 9532 MW mobil yg sama dipakai di bbrp aksi… RT  : Keberhasilan negosiasi kita. Formula E mau tuh menghormati nilai2 lokal Indonesia: tak ada sampanye sbg selebrasi kemenan… RT  :  

 

 

B 9532 MW mobil yg sama dipakai di bbrp aksi… RT  : Formula E Disebut Bikin Ancol Rugi, Geisz Ingatkan Proyek LRT: Biaya Rp6,5 T Masuk Cuma Rp500 Juta
  RT  : Nafas nyesek, mata kunang-kunang
tidur gelisah, pikiran kalut, perut mual, makan gak enak, itulah gambaran kondisi  … RT  : Formula E menyingkap Gorden, antara Petarumg dengan Pecundang. RT  : Buzzer Genjer STRESSSSSS gara2 Formula E sukses 🤣🤣

Ormas Terlarang kok demonya dijagain polisi, bukan dibubarin🤣🤣 https://… RT  : Sukses Formu

In [ ]:
# menghapus hashtag dalam tweet
getHashtags = re.compile(r"#(\w+)")
print("Tags = {0}".format(re.findall(getHashtags, T4)))

pisahtags = re.compile(r'[A-Z][^A-Z]*')

for tags in re.findall(getHashtags, T4):
    print(re.findall(pisahtags, tags))

# Mengganti hashtags dengan kata dasar pembentuknya
tagS = re.findall(getHashtags, T4)
for tag in tagS:
    proper_words = ' '.join(re.findall(pisahtags, tag))
    T5 = T4.replace('#'+tag,proper_words)

print(''.join(T5))

Output streaming akan dipotong hingga 5000 baris terakhir.

FPI Gadungan selesai acara jilbab la… RT  : "Formula E menjadi pelajaran bagi kita semua, sebelum mengadakan maka harus dilihat apakah event yang akan diadakan diminati m… RT  : Tudingan Anies Hanya Pintar Ngomong Terbantahkan Lewat Formula E
  RT  : Media nih suka aneh2..
Yg mana sih fans Formula E itu? Coba suruh mrk sbutin satu nama pmbalap Formula E tnpa googling..
Tau… RT  :  

 

 

B 9532 MW mobil yg sama dipakai di bbrp aksi… RT  : “Anggota DPRD DKI Fraksi PDIP: Jokowi Hadir Bukan Berarti Dukung Formula E”

Ini si Gilbert bencinya ke Anies sdh ke u… RT  : Buzzer Genjer STRESSSSSS gara2 Formula E sukses 🤣🤣

Ormas Terlarang kok demonya dijagain polisi, bukan dibubarin🤣🤣 https://… RT  : Formula E Dikritik Giring PSI, Wagub Ariza: Alhamdulillah, Sesuai Harapan
  RT  : BUMN tidak mau menjadi sponsor, TV swasta memboikot, tidak pakai pawang hujan tapi Formula E tetap Sukses.
Semua berkat doa Ra… RT  : Ini akibat dendam kesum

In [ ]:
# Menghilangkan non alpha-numeric (terutama emote)
T6 = (re.sub(r'[^\w]',' ',T5))
print(T6)

  Ini linknya Prof  Refly Harun   Satu di Anak Emaskan tapi  Gagal Total di MotoGP Mandalika    Satunya Lagi     Anies Baswedan di anggap Anak Pinggiran yang Sukses Besar Menggelar Event Internasional Balap Mobil Formula                       RT      Ternyata dipake juga buat demo usut formula E    RT             B 9532 MW mobil yg sama dipakai di bbrp aksi  RT    Keberhasilan negosiasi kita  Formula E mau tuh menghormati nilai2 lokal Indonesia  tak ada sampanye sbg selebrasi kemenan  RT             B 9532 MW mobil yg sama dipakai di bbrp aksi  RT    Formula E Disebut Bikin Ancol Rugi  Geisz Ingatkan Proyek LRT  Biaya Rp6 5 T Masuk Cuma Rp500 Juta   RT    Nafas nyesek  mata kunang kunang tidur gelisah  pikiran kalut  perut mual  makan gak enak  itulah gambaran kondisi    RT    Formula E menyingkap Gorden  antara Petarumg dengan Pecundang  RT    Buzzer Genjer STRESSSSSS gara2 Formula E sukses     Ormas Terlarang kok demonya dijagain polisi  bukan dibubarin   https     RT    Sukses Formu

In [ ]:
# Menanagani Slang dan Singkatan dari tweets

df=open('data/slang.txt',"r",encoding="utf-8", errors='replace')
slangS = df.readlines(); df.close()
slangS = [t.strip('\n').strip() for t in slangS]
print(slangS[:5])
# tambahkan beberapa slang di tweet yang berhasil diidentifikasi secara langsung
slangS.append("stressssss : stress")
slangS.append("dipake : dipakai")
slangS.append("amp : dan")
slangS.append("stlh : setelah")
slangS.remove("mw : now watching")
slangS.remove("mw : mau")

# pisahkan berdasarkan ':'
slangS = [t.split(":") for t in slangS]
slangS = [[k.strip(), v.strip()] for k,v in slangS]
print(slangS[:3])
slangS = {k:v for k,v in slangS}

['& : dan', '1pun : satupun', '7an : tujuan', '@ : di', 'Dr : dokter']
[['&', 'dan'], ['1pun', 'satupun'], ['7an', 'tujuan']]


In [ ]:
# Tokenisasi dan mengganti slang dan singkatan pada tweets
from textblob import TextBlob
T=''.join([str(item) for item in T6])
T = T.lower() #supaya huruf kecil semua
Tokens = TextBlob(T).words # Tokenisasi

for i,t in enumerate(Tokens):
    if t in slangS.keys():
        Tokens[i] = slangS[t]
T7 = [t for t in Tokens]
print(' '.join(T7))


ini linknya prof refly harun satu di anak emaskan tapi gagal total di motogp mandalika satunya lagi anies baswedan di anggap anak pinggiran yang sukses besar menggelar event internasional balap mobil formula rt ternyata dipakai juga buat demo usut formula e rt b 9532 mw mobil yang sama dipakai di beberapa aksi rt keberhasilan negosiasi kita formula e mau itu menghormati nilai2 lokal indonesia tidak ada sampanye sebagai selebrasi kemenan rt b 9532 mw mobil yang sama dipakai di beberapa aksi rt formula e disebut bikin ancol rugi geisz ingatkan proyek lrt biaya rp6 5 t masuk cuma rp500 juta rt nafas nyesek mata kunang kunang tidur gelisah pikiran kalut perut mual makan tidak enak itulah gambaran kondisi rt formula e menyingkap gorden antara petarumg dengan pecundang rt buzzer genjer stress gara2 formula e sukses ormas terlarang kok demonya dijagain polisi bukan dibubarin https rt sukses formula e bikin kaum pecundang kalap jakarta mendunia formulaejakartaaniesdasyat formulaejakartaaniesda

In [ ]:
# Loading Stopwords
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()

Sastrawi_StopWords_id = factory.get_stop_words()

df=open('data/stopwords_id.txt',"r",encoding="utf-8", errors='replace')
id_stop = df.readlines()
df.close()
id_stop = [t.strip().lower() for t in id_stop]

N = 10
print(Sastrawi_StopWords_id[:N])
print(id_stop[:N])
print(len(Sastrawi_StopWords_id), len(id_stop))

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua']
['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir']
126 758


In [ ]:
# Mengubah list stopwords ke bentuk set karena di Python lebih cepat cek existence di set daripada list
Sastrawi_StopWords_id = set(Sastrawi_StopWords_id)
id_stop = set(id_stop)

# Stopwords
from textblob import TextBlob

T=' '.join([str(item) for item in T7])
id_stop.add('rt')
id_stop.add('formula')
id_stop.add('e')
id_stop.add('formulae')
id_stop.add('formulaejakarta')
id_stop.add('🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨🇲🇨')
id_stop.add('https')
Tokens = TextBlob(T).words

T8 = [t for t in Tokens if t not in id_stop] # Sastrawi_StopWords_id Personal_StopWords_en Personal_StopWords_id
T9=' '.join(T8)
print(T9)
# Catatan: Selalu lakukan Stopword filtering setelah tokenisasi (dan normalisasi).

linknya prof refly harun anak emaskan gagal total motogp mandalika satunya anies baswedan anggap anak pinggiran sukses menggelar event internasional balap mobil dipakai demo usut b 9532 mw mobil dipakai aksi keberhasilan negosiasi menghormati nilai2 lokal indonesia sampanye selebrasi kemenan b 9532 mw mobil dipakai aksi bikin ancol rugi geisz ingatkan proyek lrt biaya rp6 5 t masuk rp500 juta nafas nyesek mata kunang kunang tidur gelisah pikiran kalut perut mual makan enak gambaran kondisi menyingkap gorden petarumg pecundang buzzer genjer stress gara2 sukses ormas terlarang demonya dijagain polisi dibubarin sukses bikin kaum pecundang kalap jakarta mendunia formulaejakartaaniesdasyat formulaejakartaaniesdasyat wow hasil transaksi penjualan umkm jakarta memuaskan langsung mengajari generasi muda sportif keberhasilan lihat keramaian hingar bingar pengurus negeri puas perhelatan cobaan ujian berhenti berlanjut sukses dijegal ki hihihi jorok maennya jatuhin anies anies sukses jis sirkuit 

In [ ]:
# Menyimpan hasil preprocessing
import taudataEDA02CTA as eda
filename = "HasilPP_Tweets_FormulaE.txt"
eda.saveTweets(T8, file=filename, type_="plain")

"Tweets Saved: ".format(filename) # Silahkan cek di folder "Data", pindahkan ke Desktop untukk memudahkan tahap selanjutnya

'Tweets Saved: '